# USER INTERFACE
Implementation for a user interface with various commands:
- modify the speed of the roboto, both angular and linear.
- decide the robot behaviour, random or user controlled
- directional control of the robot
In later cells I display various informations such as:
- robot position and orientation
- robot linear and angular speed
- number of reached and aborted goals
- time to reach each goal

# CODE

## Libraries
The first block is where I import the various libraries I will need in the code.

In [1]:

import rospy
import time
import jupyros as jr
import matplotlib
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import math
import actionlib
import actionlib.msg

%matplotlib widget

from matplotlib.animation import FuncAnimation
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from std_msgs.msg import String, Float64, Bool, Float32
from rt2_assignment1.srv import Command
from ipywidgets import interact,interactive,fixed,interact_manual
from geometry_msgs.msg import Twist

## Initialization
Here I initialiaze the ros node, one service client (/user_interface), two publishers(/vel and /cmd_vel) and various global variables.

In [2]:
rospy.init_node('user_interface')
client=rospy.ServiceProxy('/user_interface', Command)
vel_p = rospy.Publisher('/vel', Twist, queue_size = 1)
com_p = rospy.Publisher('/cmd_vel', Twist, queue_size = 1)
vel = Twist()
speed = Twist()
start = time.time()
#finished = interrupted = 0
t = []
t1 = []
x = []
y = []
linear_vel = []
angular_vel = []
linear_speed = []
angular_speed = []
cycles_interrupt = [0,0]
vel_lin = 0
vel_ang = 0
check_random = False
reached=0
interrupted=0

## Buttons and Sliders
### Buttons' and Sliders' functions definition
THis is the various functions I run when the user clicks the corrispondent button

In [3]:
# Random behaviour start
def Start_RB(p):
    global check_random
    check_random = True
    client('start')

# Random behaviour stop
def Stop_RB(p):
    global check_random
    check_random = False
    client('stop')

# To move ahead
def Straight(p):
    global check_random, speed, vel
    if check_random:
        check_random = False
        client('stop')
    Stop(1)
    vel.linear.x = speed.linear.x

    com_p.publish(vel)

# To move back
def Back(p):
    global check_random, speed, vel
    if check_random:
        check_random = False
        client('stop')
    Stop(1)
    vel.linear.x = - speed.linear.x

    com_p.publish(vel)

# To move left
def Left(p):
    global check_random, speed, vel
    if check_random:
        check_random = False
        client('stop')
    Stop(1)
    vel.angular.z = - speed.angular.z

    com_p.publish(vel)

# To move right
def Right(p):
    global check_random, speed, vel
    if check_random:
        check_random = False
        client('stop')
    Stop(1)
    vel.angular.z = speed.angular.z

    com_p.publish(vel)

# To stop
def Stop(p):
    vel.linear.x = 0
    vel.angular.z = 0
    com_p.publish(vel)

# To set linear vel
def Set_lin_vel(vel):
    global speed
    speed.linear.x = vel
    vel_p.publish(speed)
    return vel

# To set angular vel
def Set_ang_vel(vel):
    global speed
    speed.angular.z = vel
    vel_p.publish(speed)
    return vel

### Buttons initialization
This block is where all the buttons are initialzed and defined.

In [4]:
# Button to start the random behaviour
RB_Start=widgets.Button(
    description='Start RB',
    style=widgets.ButtonStyle(button_color='green'),
    layout=widgets.Layout(width='auto', grid_area='start_rb')
)

# Button to stop the random behaviour
RB_Stop=widgets.Button(
    description='Stop RB',
    style=widgets.ButtonStyle(button_color='red'),
    layout=widgets.Layout(width='auto', grid_area='stop_rb')
)

# Button to move ahead
Go_Straight=widgets.Button(
    description='Go Straight',
    style=widgets.ButtonStyle(button_color='white'),
    layout=widgets.Layout(width='auto', grid_area='straight')
)

# Button to move back
Go_Back=widgets.Button(
    description='Back',
    style=widgets.ButtonStyle(button_color='white'),
    layout=widgets.Layout(width='auto', grid_area='back')
)

# Button to move left
Go_Left=widgets.Button(
    description='Left',
    style=widgets.ButtonStyle(button_color='white'),
    layout=widgets.Layout(width='auto', grid_area='left')
)

# Button to move right
Go_Right=widgets.Button(
    description='Right',
    style=widgets.ButtonStyle(button_color='white'),
    layout=widgets.Layout(width='auto', grid_area='right')
)

# Button to stop
Button_stop=widgets.Button(
    description='Stop',
    style=widgets.ButtonStyle(button_color='orange'),
    layout=widgets.Layout(width='auto', grid_area='stop')
)


### Buttons' function callbacks
In this block I link each button with the proper function

In [5]:
RB_Start.on_click(Start_RB)
RB_Stop.on_click(Stop_RB)
Go_Straight.on_click(Straight)
Go_Back.on_click(Back)
Go_Left.on_click(Left)
Go_Right.on_click(Right)
Button_stop.on_click(Stop)

### Sliders and Button position
In this block I initialize the sliders and position the button on the screen. The interface should be understandble enough as each button and slider is named properly.

In [6]:
# Slider for linear velocity
print("Value of the linear velocity: ")
interact(Set_lin_vel, vel =widgets.FloatSlider(min=0.0, max=5.0, step=0.1, value=1.0))

# Slider for angular velocity
print("Value of the angular velocity:")
interact(Set_ang_vel, vel =widgets.FloatSlider(min=0.0, max=5.0, step=0.1, value=1.0))

# box to position the buttons.
widgets.GridBox(children=[RB_Start, RB_Stop, Go_Straight, Go_Back, Go_Left, Go_Right, Button_stop],
                layout=widgets.Layout(width='100%',grid_template_rows='auto auto',
                grid_template_columns='12% 12% 12% 12% 12% 12% ',
                grid_template_areas='''"start_rb . . straight . stop ""stop_rb . left back right ."''')
               )

Value of the linear velocity: 


interactive(children=(FloatSlider(value=1.0, description='vel', max=5.0), Output()), _dom_classes=('widget-int…

Value of the angular velocity:


interactive(children=(FloatSlider(value=1.0, description='vel', max=5.0), Output()), _dom_classes=('widget-int…

GridBox(children=(Button(description='Start RB', layout=Layout(grid_area='start_rb', width='auto'), style=Butt…

## Subscribers initialization and callback definition
These subscribers are fundamental to display the data in the next few blocks. I read the value from /cmd_vel, /target and /odom and, using the callback functions I saved the needed values on global variables.

In [7]:
from tf import transformations


# I need this function to make sure every angle is in the [-pi, pi] interval.
def normalize_angle(angle):
    if(math.fabs(angle) > math.pi):
        angle = angle - (2 * math.pi * angle) / (math.fabs(angle))
    return angle

# callback function for the /odom subscriber
def clbk_odom(msg):
    global position_
    global yaw_
    global t1, elapsed_time, x, y, linear_vel, linear_speed, angular_vel, angular_speed, direction_x, direction_y
    # how much time has passed since che start of the script
    elapsed_time = time.time() - start
    
    # I limit the data collected to 500 samples as more is not usefull and slows the program
    # modify here if you want to change the time interval 
    if len(t1)>1000:
        t1.pop(0)
        x.pop(0)
        y.pop(0)
        linear_vel.pop(0)
        linear_speed.pop(0)
        angular_vel.pop(0)
        angular_speed.pop(0)
    
    position_ = msg.pose.pose.position
    velocity_ = msg.twist.twist

    # here I add the various data to different global arrays.
    t1.append(elapsed_time)
    x.append(position_.x)
    y.append(position_.y)
    linear_vel.append(vel_lin)
    angular_vel.append(vel_ang)
    linear_speed.append(velocity_.linear.x)
    angular_speed.append(velocity_.angular.z)
    
    # here I have the orientation as a quaternion and i modify it in eulerian angles
    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    yaw_ = euler[2]

    # angle normalization and saving the orientation
    norm_yaw = normalize_angle(yaw_)
    direction_x = math.cos(norm_yaw)
    direction_y = math.sin(norm_yaw)

def destination_clbk(msg):
    #print("dentro destination")
    global reached, interrupted, t, cycles_interrupt
    #print("inherebitch")
    # using the if cycle I add reached or aborted goals.
    if msg.data == -1:
        interrupted += 1
        #print("1")
    else: 
        reached += 1
        #print("2")
        # here is where I save the time to reach the goal
        t.append(msg.data)
    #print("target")
    cycles_interrupt = [reached, interrupted]

# /cmd_vel callback
def velocity_clbk(msg):
    global vel_lin, vel_ang
    vel_lin = msg.linear.x
    vel_ang = msg.angular.z


# Subscriber initialization
com_s = rospy.Subscriber('/cmd_vel', Twist, velocity_clbk)
odom_s = rospy.Subscriber('/odom', Odometry, clbk_odom)
dest_s = rospy.Subscriber('/destination', Float32, destination_clbk)
#jr.subscribe('/destination', Float32, destination_clbk)
#jr.subscribe('/odom', Odometry, clbk_odom)
#jr.subscribe('/cmd_vel', Twist, velocity_clbk)
print("finished")

finished


## PLOTS
### Velocities
Here I plot the graphs for linear and angular velocity, both the request and the actual velocity.
As we run the code we can see that:
- the linear velocities mostly match, outside of a little delay in actuation and some oscillations in the beginning
- the angular velocities have, as expected, different sings due to different reference frames (clockwise for /cmd_vel and couter clockwise for /odom). THe magnitude is alse different due to the upper and lower bound in the go_to_point.py script.

In [8]:
# defining the plot for the linear velocities.
fig1,axis1=plt.subplots()
axis1.set_ylim((-5,5))
#axis1.set_xlim((0.15))
linea=axis1.plot([],[],lw=1.5,color='black',label='Actual Velocity')[0]
lineb=axis1.plot([],[],lw=1.5,color='red',label='Input Velocity')[0]
lines1,=axis1.plot([],[], lw=1.5)
axis1.legend()
axis1.set_title("Linear Velocity")
axis1.set_xlabel("Time[s]")
axis1.set_ylabel("Nominal Value [m/s]")
lines1_vector=[]
lines1_vector.append(linea)
lines1_vector.append(lineb)

# defining the plot for the angular velocities.
fig2,axis2=plt.subplots()
axis2.set_ylim((-5,5))
#axis2.set_xlim((0.15))
linec=axis2.plot([],[],lw=1.5,color='black',label='Actual Velocity')[0]
lined=axis2.plot([],[],lw=1.5,color='red',label='Input Velocity')[0]
lines2,=axis2.plot([],[], lw=1.5)
axis2.legend()
axis2.set_title("Angular Velocity")
axis2.set_xlabel("Time[s]")
axis2.set_ylabel("Nominal Value [rad/s]")
lines2_vector=[]
lines2_vector.append(linec)
lines2_vector.append(lined)

# plot 1 initialization
def init1():
    lines1.set_data([],[])

# plot 2 initialization
def init2():
    lines2.set_data([],[])

# plot 1 update
def anim1(b):
    axis1.set_xlim((elapsed_time-5, elapsed_time+5))
    vector_x = [t1, t1]
    vector_y = [linear_speed, linear_vel]

    for i,lines1 in enumerate(lines1_vector):
        lines1.set_data(vector_x[i], vector_y[i])
    return (lines1_vector)

# plot 2 update
def anim2(a):
    axis2.set_xlim((elapsed_time-5, elapsed_time+5))
    vector_x = [t1, t1]
    vector_y = [angular_speed, angular_vel]

    for j,lines2 in enumerate(lines2_vector):
        lines2.set_data(vector_x[j], vector_y[j])
    return (lines2_vector)

animation1=animation.FuncAnimation(fig1, anim1, init_func = init1, frames=100, interval=100, blit=True)
animation2=animation.FuncAnimation(fig2, anim2, init_func = init2, frames=100, interval=100, blit=True)





Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Goals Reached or Aborted
Here, when the user runs the code, is where I plot the reached and aborted goal using a column chart.

In [9]:
# plot definition
fig3,axis3 =  plt.subplots()
legend = ['Reached', 'Interrupted']
#axis3.bar(legend,stats,color = 'r', width = 0.5)
axis3.set_title("Reached and Interrupted goal")

#plot inizialization
def init3():
    axis3.bar(legend,cycles_interrupt,color = 'blue', width = 0.6)

#plot update
def anim3(i):
    global axis3
    axis3.bar(legend,cycles_interrupt,color = 'blue', width = 0.6)

animation3=animation.FuncAnimation(fig3, anim3, init_func=init3, frames=100, interval=100, blit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Time
Here I plot the time to reach each goal, using a histogram. The X axis represents time and the Y axis the number of goals reached in the same time.

In [10]:
fig4,axis4 = plt.subplots()
#axis4.hist(t, 10, label='Time to reach the goal', color='blue', lw=10)
axis4.legend()
axis4.set_title("Time passed before the goal reachment")
axis4.set_xlabel("Time[s]")

def init4():
    axis4.hist(t, 10, label='Time to reach the goal', color='blue', lw=10)

def anim4(i):
    global t, axis4
    axis4.hist(t, 10, label='Time to reach the goal', color='blue', lw=10)
    
animation4 = animation.FuncAnimation(fig4, anim4, init_func=init4, frames=100, interval=100, blit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Position and Orientation
Here is where I plot the position and Orientation. The position saves a trajectory (plotted using the last 500 positions as having bigger intervals would slow the program). The orientation is represented as an arrow.

In [11]:
# defining both plots. Two are needed as i have to clear the orientation arrow with every iteration but I want to have the trajectory plotted to have less computations.
fig5,axis5 = plt.subplots()
fig6,axis6 = plt.subplots()

axis5.set_xlim((-10,10))
axis5.set_ylim((-10,10))
linee,=axis5.plot([],[], lw=1.5, color='green', label='Position')
axis5.legend()
axis5.set_title("Pos")
#axis6.quiver(0,0,dir_x,dir_y,label="Orientation")
#axis6.legend()
#axis6.set_title("Orientation")

# Position plot initialization
def init5():
    linee.set_data([],[])

# Orientation plot initialization
def init6():
    axis6.quiver(0,0,direction_x,direction_y,label="Orientation")

# Position plot update
def anim5(i):
    global x, y
    linee.set_data(x,y)
    return (linee)

# Orientation plot update
def anim6(i):
    axis6.clear()
    axis6.quiver(0,0,direction_x,direction_y,label="Orientation")
    axis6.legend()
    axis6.set_title("Orien")
    return(linee)

animation5 = animation.FuncAnimation(fig5, anim5, init_func=init5, frames=100, interval=100, blit=True)
animation6 = animation.FuncAnimation(fig6, anim6, init_func=init6, frames=1, interval=100, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
print(cycles_interrupt)

[0, 0]


[0, 0]
